# Prompt Patterns - Part 3

This notebook demonstrates advanced prompt patterns, including success and failure cases.

## Patterns Covered
1. Output Customization
2. Input Semantics
3. Chain Prompting
4. Meta Prompting

## Setup

In [1]:
!pip install openai langchain langchain-community
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the OpenAI client
client = OpenAI(api_key="skfake")

# Initialize ChatOpenAI for LangChain
chat_model = ChatOpenAI(
    temperature=0,
    openai_api_key=client.api_key,
    model_name="gpt-3.5-turbo"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


<ipython-input-1-309068b2fe03>:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(


## Helper Functions

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    """Get completion from OpenAI API using the latest client"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

def test_template(template, test_cases):
    """Test a template with multiple cases"""
    print("Template:\n")
    print(template)
    print("\nTest Cases:\n")
    for case in test_cases:
        print(f"Input: {case}")
        prompt = template.format(text=case)
        result = get_completion(prompt)
        print(f"Output: {result}")
        print("-"*75)

def chain_prompts(input_text):
    """Execute a chain of prompts with dependencies"""

    # Step 1: Extract key concepts
    concept_template = """
    Extract key concepts from the text:
    {text}

    Provide:
    1. Main concepts
    2. Related terms
    3. Domain/field
    """
    concepts = get_completion(concept_template.format(text=input_text))

    # Step 2: Analyze relationships
    relationship_template = """
    Analyze relationships between these concepts:
    {text}

    Provide:
    1. Direct connections
    2. Hierarchical relationships
    3. Dependencies
    """
    relationships = get_completion(relationship_template.format(text=concepts))

    # Step 3: Generate insights
    insight_template = """
    Generate insights based on:
    Original text: {text}
    Key concepts: {concepts}
    Relationships: {relationships}

    Provide:
    1. Key findings
    2. Implications
    3. Recommendations
    4. Next steps
    """
    insights = get_completion(
        insight_template.format(
            text=input_text,
            concepts=concepts,
            relationships=relationships
        )
    )

    return {
        "concepts": concepts,
        "relationships": relationships,
        "insights": insights
    }

## 1. Output Customization Pattern

### Success Cases

In [3]:
output_customization_template = """
Generate a report about the given topic in multiple formats:

1. Executive Summary
   - Overview (2-3 sentences)
   - Key points (3-4 bullets)
   - Business impact

2. Detailed Analysis
   - Current state
   - Trends and patterns
   - Future implications
   - Recommendations

3. Technical Documentation
   - Methodology
   - Data sources
   - Detailed findings
   - Technical notes

Topic: {text}
"""

success_cases = [
    "Impact of AI on healthcare",
    "Renewable energy adoption trends",
    "Future of remote work"
]

print("Success Cases:")
test_template(output_customization_template, success_cases)

Success Cases:
Template:


Generate a report about the given topic in multiple formats:

1. Executive Summary
   - Overview (2-3 sentences)
   - Key points (3-4 bullets)
   - Business impact

2. Detailed Analysis
   - Current state
   - Trends and patterns
   - Future implications
   - Recommendations

3. Technical Documentation
   - Methodology
   - Data sources
   - Detailed findings
   - Technical notes

Topic: {text}


Test Cases:

Input: Impact of AI on healthcare
Output: Executive Summary:
AI has revolutionized the healthcare industry by improving diagnosis accuracy, personalized treatment plans, and operational efficiency. The key points include increased patient outcomes, reduced healthcare costs, improved decision-making, and enhanced patient experience. The business impact of AI in healthcare includes better resource allocation, improved patient satisfaction, and increased revenue potential.

Detailed Analysis:
- Current state: AI is being used in healthcare for tasks like me

### Failure Case

In [4]:
failure_cases = [
    "Things",  # Too vague
    "ABC"      # Non-descriptive
]

print("Failure Cases:")
test_template(output_customization_template, failure_cases)

Failure Cases:
Template:


Generate a report about the given topic in multiple formats:

1. Executive Summary
   - Overview (2-3 sentences)
   - Key points (3-4 bullets)
   - Business impact

2. Detailed Analysis
   - Current state
   - Trends and patterns
   - Future implications
   - Recommendations

3. Technical Documentation
   - Methodology
   - Data sources
   - Detailed findings
   - Technical notes

Topic: {text}


Test Cases:

Input: Things
Output: Executive Summary:
Overview: This report provides an analysis of various things and their impact on businesses. It explores current trends, patterns, and future implications related to things.

Key points:
- Understanding consumer preferences for certain things can help businesses tailor their products and services.
- Trends in the market for certain things can indicate potential opportunities for growth or diversification.
- Businesses need to stay updated on the latest developments in the things industry to remain competitive.

Bu

### Fix: Structured Output Generation

In [5]:
robust_output_template = """
Generate a comprehensive multi-format report following these guidelines:

1. Topic Analysis
   - Main concept
   - Key aspects
   - Current relevance

2. Output Formats

   Executive Summary:
   - Overview sentence
   - Key findings
   - Business impact

   Detailed Analysis:
   - Current state
   - Trends and patterns
   - Future implications
   - Recommendations

   Technical Documentation:
   - Methodology
   - Data sources
   - Detailed findings
   - Technical considerations

For vague or unclear topics:
1. Request clarification
2. Suggest specific angles
3. Provide example topics
4. List required information

Topic: {text}
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_output_template, test_cases)

Testing Robust Template:
Template:


Generate a comprehensive multi-format report following these guidelines:

1. Topic Analysis
   - Main concept
   - Key aspects
   - Current relevance

2. Output Formats

   Executive Summary:
   - Overview sentence
   - Key findings
   - Business impact

   Detailed Analysis:
   - Current state
   - Trends and patterns
   - Future implications
   - Recommendations

   Technical Documentation:
   - Methodology
   - Data sources
   - Detailed findings
   - Technical considerations

For vague or unclear topics:
1. Request clarification
2. Suggest specific angles
3. Provide example topics
4. List required information

Topic: {text}


Test Cases:

Input: Impact of AI on healthcare
Output: Topic Analysis:

Main concept: The impact of Artificial Intelligence (AI) on healthcare refers to the use of AI technologies such as machine learning, natural language processing, and robotics to improve healthcare delivery, diagnosis, treatment, and patient outcomes.



## 2. Input Semantics Pattern

### Success Cases

In [6]:
input_semantics_template = """
Analyze the semantic structure of the following text:

1. Content Analysis
   - Main concepts
   - Key entities
   - Technical terms
   - Domain context

2. Semantic Structure
   - Subject-predicate relationships
   - Concept hierarchies
   - Temporal relationships
   - Causal connections

3. Implicit Information
   - Assumptions
   - Implications
   - Inferences
   - Context requirements

Text: {text}
"""

success_cases = [
    "The new AI model outperformed previous benchmarks in natural language processing tasks.",
    "Sustainable energy solutions are becoming more cost-effective as technology advances.",
    "Remote work has transformed how companies approach team collaboration and productivity."
]

print("Success Cases:")
test_template(input_semantics_template, success_cases)

Success Cases:
Template:


Analyze the semantic structure of the following text:

1. Content Analysis
   - Main concepts
   - Key entities
   - Technical terms
   - Domain context

2. Semantic Structure
   - Subject-predicate relationships
   - Concept hierarchies
   - Temporal relationships
   - Causal connections

3. Implicit Information
   - Assumptions
   - Implications
   - Inferences
   - Context requirements

Text: {text}


Test Cases:

Input: The new AI model outperformed previous benchmarks in natural language processing tasks.
Output: Semantic Structure Analysis:

1. Content Analysis:
   - Main concepts: AI model, benchmarks, natural language processing tasks
   - Key entities: AI model, benchmarks
   - Technical terms: AI model, natural language processing tasks
   - Domain context: Artificial Intelligence, Natural Language Processing

2. Semantic Structure:
   - Subject-predicate relationships: The AI model (subject) outperformed (predicate) previous benchmarks.
   - Concep

### Failure Case

In [7]:
failure_cases = [
    "OK",       # Too brief
    "12345"     # Non-semantic content
]

print("Failure Cases:")
test_template(input_semantics_template, failure_cases)

Failure Cases:
Template:


Analyze the semantic structure of the following text:

1. Content Analysis
   - Main concepts
   - Key entities
   - Technical terms
   - Domain context

2. Semantic Structure
   - Subject-predicate relationships
   - Concept hierarchies
   - Temporal relationships
   - Causal connections

3. Implicit Information
   - Assumptions
   - Implications
   - Inferences
   - Context requirements

Text: {text}


Test Cases:

Input: OK
Output: The text outlines the components of content analysis, semantic structure, and implicit information. It discusses the main concepts, key entities, technical terms, and domain context in content analysis. It also delves into subject-predicate relationships, concept hierarchies, temporal relationships, and causal connections in semantic structure. Implicit information such as assumptions, implications, inferences, and context requirements are also highlighted. The text appears to be a brief overview or outline of these analytical p

### Fix: Comprehensive Semantic Analysis

In [8]:
robust_semantics_template = """
Perform a comprehensive semantic analysis following this framework:

1. Content Analysis
   - Main topics
   - Key concepts
   - Named entities
   - Technical terms

2. Semantic Structure
   - Subject-predicate relationships
   - Concept hierarchies
   - Temporal relationships
   - Causal connections

3. Contextual Analysis
   - Domain context
   - Background knowledge
   - Cultural references
   - Temporal context

4. Implicit Information
   - Assumptions
   - Implications
   - Inferences
   - Unstated connections

For invalid or unclear input:
1. Note limitations
2. Suggest improvements
3. Provide examples
4. List requirements

Text: {text}
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_semantics_template, test_cases)

Testing Robust Template:
Template:


Perform a comprehensive semantic analysis following this framework:

1. Content Analysis
   - Main topics
   - Key concepts
   - Named entities
   - Technical terms

2. Semantic Structure
   - Subject-predicate relationships
   - Concept hierarchies
   - Temporal relationships
   - Causal connections

3. Contextual Analysis
   - Domain context
   - Background knowledge
   - Cultural references
   - Temporal context

4. Implicit Information
   - Assumptions
   - Implications
   - Inferences
   - Unstated connections

For invalid or unclear input:
1. Note limitations
2. Suggest improvements
3. Provide examples
4. List requirements

Text: {text}


Test Cases:

Input: The new AI model outperformed previous benchmarks in natural language processing tasks.
Output: 1. Content Analysis:
   - Main topics: AI model, natural language processing tasks
   - Key concepts: AI performance, benchmarks, natural language processing
   - Named entities: AI model
   - T

## 3. Chain Prompting Pattern

### Success Cases

In [9]:
success_cases = [
    "AI and machine learning are transforming healthcare through improved diagnostics and personalized treatment.",
    "Renewable energy adoption is accelerating as solar and wind power become more cost-effective."
]

print("Success Cases:")
for case in success_cases:
    print(f"\nInput: {case}")
    result = chain_prompts(case)
    print("Result:")
    print(result)
    print("-"*75)

Success Cases:

Input: AI and machine learning are transforming healthcare through improved diagnostics and personalized treatment.
Result:
{'concepts': '1. Main concepts:\n- AI (Artificial Intelligence)\n- Machine learning\n- Healthcare\n- Diagnostics\n- Personalized treatment\n\n2. Related terms:\n- Transformation\n- Improved\n- Technology\n- Data analysis\n- Predictive modeling\n\n3. Domain/field:\n- Healthcare\n- Technology\n- Data science', 'relationships': '1. Direct connections:\n- AI and machine learning are used in healthcare for diagnostics and personalized treatment.\n- Data analysis and predictive modeling are essential components of AI and machine learning in healthcare.\n- Technology plays a crucial role in enabling AI and machine learning applications in healthcare.\n\n2. Hierarchical relationships:\n- AI and machine learning are overarching concepts that encompass technologies like data analysis and predictive modeling, which are used in healthcare for diagnostics and p

### Failure Case

In [10]:
failure_cases = [
    "The thing does stuff",  # Too vague
    "A B C D"                # No meaningful content
]

print("Failure Cases:")
for case in failure_cases:
    print(f"\nInput: {case}")
    result = chain_prompts(case)
    print("Result:")
    print(result)
    print("-"*75)

Failure Cases:

Input: The thing does stuff
Result:
{'concepts': '1. Main concepts:\n- Thing\n- Stuff\n\n2. Related terms:\n- Object\n- Action\n- Activity\n\n3. Domain/field:\n- General concept, could apply to various fields such as technology, science, or everyday life.', 'relationships': '1. Direct connections:\n- Thing and Stuff are both general terms used to refer to objects or items.\n- Object is a related term that can be considered a specific type of thing or stuff.\n- Action and Activity are related terms that can involve things or stuff being used or interacted with.\n\n2. Hierarchical relationships:\n- Thing and Stuff are at the same level in terms of hierarchy, both being general concepts.\n- Object can be seen as a subset of Thing or Stuff, as it refers to a specific item or entity.\n- Action and Activity can be seen as processes or events that involve things or stuff, making them dependent on the presence of objects.\n\n3. Dependencies:\n- Actions and Activities depend on 

### Fix: Robust Chain Prompting

In [11]:
def robust_chain_prompts(input_text):
    """Execute a robust chain of prompts with validation"""

    # Step 1: Validate input
    validation_template = """
    Analyze the input text for meaningful content:
    {text}

    Provide:
    1. Content quality assessment
    2. Identified issues
    3. Suggested improvements
    4. Required clarifications
    """
    validation = get_completion(validation_template.format(text=input_text))

    # Step 2: Extract and categorize concepts
    concept_template = """
    Extract and categorize concepts from the text:
    {text}

    Provide:
    1. Primary concepts
    2. Supporting concepts
    3. Technical terms
    4. Domain context
    """
    concepts = get_completion(concept_template.format(text=input_text))

    # Step 3: Analyze relationships and dependencies
    relationship_template = """
    Analyze relationships in this concept structure:
    {text}

    Provide:
    1. Concept hierarchies
    2. Causal relationships
    3. Cross-concept links
    4. Recurring patterns
    """
    relationships = get_completion(relationship_template.format(text=concepts))

    # Step 4: Generate insights and recommendations
    insight_template = """
    Generate comprehensive insights based on:
    Original text: {text}
    Concept analysis: {concepts}
    Relationship analysis: {relationships}

    Provide:
    1. Key insights
    2. Implications
    3. Recommendations
    4. Next steps
    """
    insights = get_completion(
        insight_template.format(
            text=input_text,
            concepts=concepts,
            relationships=relationships
        )
    )

    return {
        "validation": validation,
        "concepts": concepts,
        "relationships": relationships,
        "insights": insights
    }

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Chain:")
for case in test_cases:
    print(f"\nInput: {case}")
    result = robust_chain_prompts(case)
    print("Result:")
    print(result)
    print("-"*75)

Testing Robust Chain:

Input: AI and machine learning are transforming healthcare through improved diagnostics and personalized treatment.
Result:
{'validation': '1. Content quality assessment:\nThe content is informative and relevant, highlighting the impact of AI and machine learning in the healthcare industry.\n\n2. Identified issues:\nThere are no major issues with the content provided.\n\n3. Suggested improvements:\nIt would be beneficial to provide specific examples or case studies to further illustrate how AI and machine learning are being used in healthcare for improved diagnostics and personalized treatment.\n\n4. Required clarifications:\nNone. The statement is clear and concise.', 'concepts': '1. Primary concepts:\n- AI\n- Machine learning\n- Healthcare\n- Diagnostics\n- Personalized treatment\n\n2. Supporting concepts:\n- Transformation\n- Improved outcomes\n- Technology\n- Data analysis\n\n3. Technical terms:\n- Artificial Intelligence (AI)\n- Machine Learning\n- Diagnosti

## 4. Meta Prompting Pattern

### Success Cases

In [12]:
meta_prompt_template = """
Generate an optimal prompt for the following task.

Consider:
1. Required information
2. Format specifications
3. Examples needed
4. Constraints

Task: {text}

Provide:
1. Prompt template
2. Component explanations
3. Usage guidelines
4. Example outputs
"""

success_cases = [
    "Create a data analysis report",
    "Generate product descriptions",
    "Write technical documentation"
]

print("Success Cases:")
test_template(meta_prompt_template, success_cases)

Success Cases:
Template:


Generate an optimal prompt for the following task.

Consider:
1. Required information
2. Format specifications
3. Examples needed
4. Constraints

Task: {text}

Provide:
1. Prompt template
2. Component explanations
3. Usage guidelines
4. Example outputs


Test Cases:

Input: Create a data analysis report
Output: Prompt template:
Create a data analysis report that includes an analysis of [required information] using [format specifications]. Include [number] examples to support your findings and adhere to the following constraints: [constraints].

Component explanations:
- Required information: Specify the data or dataset that needs to be analyzed, such as sales figures, customer demographics, website traffic, etc.
- Format specifications: Define the format in which the report should be presented, such as a written document, a presentation, a spreadsheet, etc.
- Examples needed: Determine the number of examples or case studies that should be included to illustra

### Failure Case

In [13]:
failure_cases = [
    "Do something",  # Too vague
    "Make it"        # Unclear task
]

print("Failure Cases:")
test_template(meta_prompt_template, failure_cases)

Failure Cases:
Template:


Generate an optimal prompt for the following task.

Consider:
1. Required information
2. Format specifications
3. Examples needed
4. Constraints

Task: {text}

Provide:
1. Prompt template
2. Component explanations
3. Usage guidelines
4. Example outputs


Test Cases:

Input: Do something
Output: Prompt template:
Please [action] [object] using [method] and provide [specific information] in [specified format]. Make sure to adhere to [constraints].

Component explanations:
1. [action]: The specific action that needs to be performed (e.g. analyze, calculate, create).
2. [object]: The subject or object on which the action will be performed (e.g. data, report, graph).
3. [method]: The approach or technique to be used in performing the action (e.g. statistical analysis, visualization tool).
4. [specific information]: The required details or data that should be included in the output.
5. [specified format]: The format in which the output should be presented (e.g. tabl

### Fix: Structured Meta Prompting

In [14]:
robust_meta_template = """
Design an optimal prompt following this framework:

1. Task Analysis
   - Core objective
   - Required inputs
   - Expected outputs
   - Success criteria

2. Prompt Components
   - Context setting
   - Input parameters
   - Format requirements
   - Constraints

3. Enhancement Features
   - Examples
   - Edge cases
   - Error handling
   - Validation rules

4. Usage Guidelines
   - Best practices
   - Common pitfalls
   - Optimization tips
   - Testing approach

For unclear tasks:
1. Request clarification
2. Suggest specific goals
3. Provide examples
4. List requirements

Task: {text}

Provide:
1. Task analysis
2. Prompt design
3. Example usage
4. Best practices
"""

# Test with both success and failure cases
test_cases = success_cases + failure_cases
print("Testing Robust Template:")
test_template(robust_meta_template, test_cases)

Testing Robust Template:
Template:


Design an optimal prompt following this framework:

1. Task Analysis
   - Core objective
   - Required inputs
   - Expected outputs
   - Success criteria

2. Prompt Components
   - Context setting
   - Input parameters
   - Format requirements
   - Constraints

3. Enhancement Features
   - Examples
   - Edge cases
   - Error handling
   - Validation rules

4. Usage Guidelines
   - Best practices
   - Common pitfalls
   - Optimization tips
   - Testing approach

For unclear tasks:
1. Request clarification
2. Suggest specific goals
3. Provide examples
4. List requirements

Task: {text}

Provide:
1. Task analysis
2. Prompt design
3. Example usage
4. Best practices


Test Cases:

Input: Create a data analysis report
Output: Task Analysis:
- Core objective: To create a comprehensive data analysis report based on a given dataset.
- Required inputs: Dataset to be analyzed, analysis tools/software, report template.
- Expected outputs: Detailed data analysis